In [1]:
import numpy as np
import pandas as pd 
#import matplotlib.pyplot as plt
#import seaborn as sns
pd.set_option('max_columns', 50)
####model libraries ####################
from sklearn.linear_model import Ridge, Lasso,SGDRegressor,LinearRegression
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,RandomForestRegressor, ExtraTreesRegressor
from sklearn.tree import ExtraTreeRegressor,DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
#########################################
from sklearn.cross_decomposition import PLSRegression

from sklearn.metrics import mean_squared_error, mean_absolute_error,make_scorer,r2_score,mean_absolute_percentage_error,explained_variance_score
from sklearn.preprocessing import MinMaxScaler

#### visulization #####
import plotly.graph_objects as go


READING DATA, INITIALIZATION

In [2]:
whole_data = pd.read_csv('C:/Users/halil/Desktop/forecast_co2/datasets/ethylene_CO.txt', delimiter='\s+',names=[
   "Time",
   "CO2_con",
   "Ethy_con",
   "sensor1",
   "sensor2",
   "sensor3",
   "sensor4",
   "sensor5",
   "sensor6",
   "sensor7",
   "sensor8",
   "sensor9",
   "sensor10",
   "sensor11",
   "sensor12",
   "sensor13",
   "sensor14",
   "sensor15",
   "sensor16",])

#normalization in range (-1,1)

In [3]:

def normalize(df,column_list):
    for col in column_list:
        feature_range = (0,1)
        min_max_scaler = MinMaxScaler(feature_range=feature_range)

        df[col] = min_max_scaler.fit_transform(df[col].values.reshape(-1,1))

    return df

function that create train test data and labels from raw dataset

In [4]:

def make_data(dataframe):
    train = dataframe[:int(len(dataframe)*0.8)]
    test = dataframe[int(len(dataframe)*0.8):]

    x_train = train.copy()
    x_test = test.copy()

    y_train = x_train.pop('CO2_con')
    y_test = x_test.pop('CO2_con')

    return x_train, x_test, y_train, y_test 

machine learning model function, to get model errors 

In [5]:
def calculate_ridge(x_train,x_test,y_train,y_test):
    model = Ridge(alpha=1) 
    model.fit(x_train,y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)

    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)



def calculate_lasso(x_train,x_test,y_train,y_test):
    model = Lasso(alpha=1) 
    model.fit(x_train,y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)

    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)

    
def calculate_linear_reg(x_train,x_test,y_train,y_test):

    model = LinearRegression()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)

def calculate_xgb(x_train,x_test,y_train,y_test):
    model = XGBRegressor()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)

def calculate_kneighbors(x_train,x_test,y_train,y_test):
    model = KNeighborsRegressor()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)

def calculate_decisiontree(x_train,x_test,y_train,y_test):
    model = DecisionTreeRegressor()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)

def calculate_randomforest(x_train,x_test,y_train,y_test):
    model = RandomForestRegressor()
    model.fit(x_train, y_train)
    train_prediction = model.predict(x_train)
    test_prediction = model.predict(x_test)
    return mean_squared_error(y_train, train_prediction), mean_squared_error(y_test, test_prediction)



In [6]:
def time_encoding(dataset):    

    time_adjusted = dataset
    index_to_time = pd.Series(dataset.index/100 )
    converted_time_data = pd.to_datetime(index_to_time, unit="s",origin="2022")
    time_adjusted["Time"] = converted_time_data.dt.round("10U")
###################### this part to extract time values from string ################################
    conversion = time_adjusted["Time"].astype(str)
    time_adjusted["hour"] = conversion.apply(lambda x:int(x[11:13]))
    time_adjusted["min"] = conversion.apply(lambda x:int(x[14:16]))
    time_adjusted["sec"] = conversion.apply(lambda x:int(x[17:19]))
    time_adjusted["100hz"] = conversion.apply(lambda x:int(x[20:22]))
    encoded = time_adjusted.drop(["Time"],axis=1)

    return encoded


In [7]:
#new_data = df.drop(["Time"],axis=1)
#co2_data = new_data.drop(["Ethy_con"],axis=1)
#eth_data = new_data.drop(["CO2_con"],axis=1)

In [8]:
#orginized_time = df.copy()
#orginized_time["Time"] = pd.to_datetime(orginized_time["Time"], unit="ms",origin="2022")
#orginized_time['Time'] = orginized_time.Time.dt.round("10U")
#time_series = df[["Time","sensor1"]]
#df = df.set_index('Time')

workflow, main codes

In [9]:
df= whole_data.copy()
onehot_encoded = time_encoding(df)
# this code for normalization before encoding
#normalized = normalize(onehot_encoded, onehot_encoded.columns[:18])
# this one for normalization of after encoding, time data also encoding too 
normalized = normalize(onehot_encoded, onehot_encoded.columns)
del onehot_encoded

In [10]:
x_train, x_test, y_train, y_test = make_data(normalized)

In [11]:
normalized

,CO2_con,Ethy_con,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6,sensor7,sensor8,sensor9,sensor10,sensor11,sensor12,sensor13,sensor14,sensor15,sensor16,hour,min,sec,100hz
0,0.0,0.0,0.000491,0.001227,0.000371,0.001109,0.002784,0.003784,0.000982,0.001003,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.702064,0.871909,0.0,0.000000,0.000000,0.000000
1,0.0,0.0,0.000893,0.001100,0.000260,0.001022,0.005408,0.002141,0.001376,0.001822,0.986465,0.987660,0.977971,0.980734,0.983000,0.976365,0.687060,0.854611,0.0,0.000000,0.000000,0.010101
2,0.0,0.0,0.003488,0.000725,0.002009,0.000969,0.001725,0.000000,0.001080,0.000756,0.978539,0.972843,0.965557,0.964841,0.961766,0.955861,0.674299,0.840640,0.0,0.000000,0.000000,0.020202
3,0.0,0.0,0.001520,0.000921,0.001469,0.001445,0.000333,0.001185,0.001621,0.001204,0.961946,0.965067,0.948013,0.948631,0.939737,0.936411,0.665944,0.824535,0.0,0.000000,0.000000,0.030303
4,0.0,0.0,0.005280,0.000558,0.001357,0.001622,0.005353,0.002235,0.000884,0.002101,0.955626,0.953650,0.936945,0.932658,0.922518,0.916777,0.653255,0.809615,0.0,0.000000,0.000000,0.040404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4208256,0.0,0.0,0.327328,0.005059,0.248945,0.244478,0.418273,0.476510,0.213309,0.216121,0.016405,0.016949,0.194900,0.135783,0.004157,0.004891,0.194397,0.184144,1.0,0.694915,0.372881,0.565657
4208257,0.0,0.0,0.330911,0.004722,0.247626,0.245140,0.421291,0.470229,0.211653,0.214567,0.016445,0.017044,0.196104,0.136931,0.003705,0.004581,0.194145,0.183464,1.0,0.694915,0.372881,0.575758
4208258,0.0,0.0,0.326269,0.005129,0.248230,0.244266,0.420937,0.471349,0.212852,0.216958,0.016362,0.016811,0.196368,0.137652,0.004180,0.004909,0.194044,0.183216,1.0,0.694915,0.372881,0.585859
4208259,0.0,0.0,0.328335,0.005242,0.248642,0.244690,0.414981,0.466759,0.212133,0.217004,0.016441,0.016862,0.195839,0.138857,0.004477,0.005073,0.193919,0.184485,1.0,0.694915,0.372881,0.595960


In [12]:
#this model work really slow 
#print(calculate_xgb(x_train, x_test, y_train, y_test))

In [13]:
print(calculate_linear_reg(x_train, x_test, y_train, y_test))

(0.0282071139357089, 0.029694051170365714)


In [ ]:
print(calculate_decisiontree(x_train,x_test,y_train,y_test))

In [14]:
print(calculate_randomforest(x_train,x_test,y_train,y_test))

In [ ]:
print(calculate_ridge(x_train,x_test,y_train,y_test))

In [5]:
async def provider(var1):
    for i in range(35):
        print(var1)

In [6]:
a = [1,2,3,4,5,6,7,8,9,0]
c = "test input"



In [7]:
provider(c)

<coroutine object provider at 0x0000026968713830>